In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
import psycopg2
#для связи с postgresql

In [3]:
import csv
from io import StringIO

In [3]:
import os
from dotenv import load_dotenv
from pathlib import Path

In [4]:
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [15]:
managers_df = pd.read_csv('managers.csv', encoding='windows-1251', sep=';')

In [16]:
managers_df

,ID,LOGIN,NAME,OFFICE
0,1,hemp,Hemp A,4
1,2,jool,Jolly A,5
2,3,hobert,Hobert F,5
3,4,frankjr,Gotty F,7
4,5,aleensr,Allen D,1
5,6,goofy,Goofy M,8
6,7,duckduck,Muddy W,8


In [17]:
managers_df.columns.tolist()

['ID', 'LOGIN', 'NAME', 'OFFICE']

In [18]:
managers_df = managers_df.rename(columns=lambda x: x.lower())

In [19]:
managers_df.columns.tolist()

['id', 'login', 'name', 'office']

In [20]:
upsale_df = pd.read_csv('upsale.csv', encoding='windows-1251', sep=';')

In [21]:
upsale_df

,ID,DT,Manager_ID,CLIENT_ID,CALL_ID,FACTOR_ID,RESULT
0,3,01.02.2017,7,123,3,2,no
1,9,03.02.2017,7,90,9,1,yes
2,5,02.02.2017,5,678,5,4,yes
3,12,04.02.2017,5,734,12,2,yes
4,6,02.02.2017,4,890,6,2,no
5,2,01.02.2017,3,67,2,1,no
6,11,04.02.2017,3,678,11,5,yes
7,7,03.02.2017,2,123,7,3,yes
8,8,03.02.2017,2,45,8,2,no
9,1,01.02.2017,1,45,1,1,yes


In [22]:
upsale_df = upsale_df.rename(columns=lambda x: x.lower())

In [23]:
upsale_df.columns.to_list()

['id', 'dt', 'manager_id', 'client_id', 'call_id', 'factor_id', 'result']

#### Заливка csv-файла в свою sql базу данных

In [5]:
# см. https://stackoverflow.com/a/55495065/4527289


def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

##### -> _local postgresql_

In [7]:
user = os.getenv('PG_USER_LOCAL')
password =  os.getenv('PG_PASSWORD_LOCAL')
hostname =  os.getenv('PG_HOST_LOCAL')
# port =  os.getenv('PG_PORT_LOCAL')
database_name =  os.getenv('PG_DATABASE_LOCAL')

In [8]:
conn_text = f'postgresql+psycopg2://{user}:{password}@{hostname}/{database_name}'
con = create_engine(conn_text)

In [ ]:
managers_df.to_sql('managers', con, index=False, if_exists='replace', method=psql_insert_copy)

In [100]:
upsale_df.to_sql('upsale', con, index=False, if_exists='replace', method=psql_insert_copy)

#### Селект к таблице своей БД

In [6]:
def select(sql):
  return pd.read_sql(sql, con=con)

In [9]:
sql = '''select * from managers t'''

In [10]:
select(sql)

,id,login,name,office
0,1,hemp,Hemp A,4
1,2,jool,Jolly A,5
2,3,hobert,Hobert F,5
3,4,frankjr,Gotty F,7
4,5,aleensr,Allen D,1
5,6,goofy,Goofy M,8
6,7,duckduck,Muddy W,8


In [89]:
sql = '''select * from upsale t'''

In [90]:
select(sql)

,id,dt,manager_id,client_id,call_id,factor_id,result
0,3,01.02.2017,7,123,3,2,no
1,9,03.02.2017,7,90,9,1,yes
2,5,02.02.2017,5,678,5,4,yes
3,12,04.02.2017,5,734,12,2,yes
4,6,02.02.2017,4,890,6,2,no
5,2,01.02.2017,3,67,2,1,no
6,11,04.02.2017,3,678,11,5,yes
7,7,03.02.2017,2,123,7,3,yes
8,8,03.02.2017,2,45,8,2,no
9,1,01.02.2017,1,45,1,1,yes


1. Отобразить сколько предложений сделал каждый из менеджеров за всю историю. 
    - джойним 2 таблицы, т.к. нужно учесть всех менеджеров, то используем full outer join
    - из временной таблицы вытаскиваем необходимые данные, группируем и сортируем.

In [14]:
sql = '''
    with count_upsales as (
    select m.id, m.login, m.name, t.manager_id 
    from upsale t
        full outer join managers m on t.manager_id = m.id)

    select t.id, t.login, t.name, count(t.manager_id) as cnt
    from count_upsales t
        group by t.id, t.login, t.name
        order by t.id

'''
select(sql)

,id,login,name,cnt
0,1,hemp,Hemp A,3
1,2,jool,Jolly A,2
2,3,hobert,Hobert F,2
3,4,frankjr,Gotty F,1
4,5,aleensr,Allen D,2
5,6,goofy,Goofy M,0
6,7,duckduck,Muddy W,2


2. Отобразить конверсию предложений в подключения для менеджеров за указанный период времени.
    - также оперируем созданной временной таблицей:
        - выбираем необходимые столбцы и формулы, делим количество строк result со значением 'yes' на общее количество строк result
        - для того, чтобы в результате получился тип данных float, приводим числитель к типу float
        - группируем и сортируем

In [94]:
sql = '''
    with count_upsales as (
    select m.id, m.login, m.name, t.result 
    from upsale t
        left join managers m on t.manager_id = m.id)

    select t.id, t.login, t.name,
        (count(t.result) filter (where t.result = 'yes')::float / count(t.result)) as convers
    from count_upsales t
        group by t.id, t.login, t.name
        order by convers desc

'''
select(sql)

,id,login,name,convers
0,5,aleensr,Allen D,1.000000
1,1,hemp,Hemp A,0.666667
2,7,duckduck,Muddy W,0.500000
3,2,jool,Jolly A,0.500000
4,3,hobert,Hobert F,0.500000
5,4,frankjr,Gotty F,0.000000


2.1. Отобразить конверсию предложений в подключения для менеджеров за определенный период времени.
    - То же самое решение, только добавлен фильтр по дате во временной таблице, потому что условие задачи отличается от предыдущей и возможно, что "вся история" != "за указанный период"

In [297]:
sql = '''
    with count_upsales as (
        select m.id, m.login, m.name, t.dt, t.result
        from upsale t
            left join managers m on t.manager_id = m.id
            where t.dt between '01.02.2017' and '03.02.2017')

    select t.id, t.login, t.name,
        (count(t.result) filter (where t.result = 'yes')::float / count(t.result)) as convers
    from count_upsales t
        group by t.id, t.login, t.name
        order by convers desc

'''
select(sql)

,id,login,name,convers
0,5,aleensr,Allen D,1.0
1,1,hemp,Hemp A,0.5
2,2,jool,Jolly A,0.5
3,7,duckduck,Muddy W,0.5
4,3,hobert,Hobert F,0.0
5,4,frankjr,Gotty F,0.0


3.  Вывести конверсии тех менеджеров, у которых было больше 100 звонков за указанный период времени.
    - В данных 12 строк, поэтому таких менеджеров не оказалось

In [80]:
sql = '''
    with count_upsales as (
    select m.id, m.login, m.name, t.result 
    from upsale t
        left join managers m on t.manager_id = m.id)

    select t.id, t.login, t.name, count(t.result) as cnt_call,
    (count(t.result) filter (where t.result = 'yes')::float / count(t.result)) as convers
    from count_upsales t
        group by t.id, t.login, t.name
        having count(t.result) > 100
        order by convers desc
'''
select(sql)


,id,login,name,cnt_call,convers


4. Вывести офисы, отсортированные в порядке убывания средней конверсии менеджеров из офиса за всю историю.
    - сначала заджойнил 2 таблицы. Т.к. речь идет о конверсии менеджеров, то решил собирать не всех менеджеров, исключил тех, кто не сделал ни одного звонка и значит по которым невозможно подсчитать конверсию. Здесь все зависит от методологии оценки офиса. Может быть другое решение, в котором отсутствие звонков учитывается при оценки эффективности офиса.
    - далее подсчитал конверсию сгруппированную по офисам.
    - в конце использовал оконную функцию подсчета среднего значения.

In [321]:
sql = '''
    with count_offices as (

        with count_result as (
        select m.office, t.result 
        from upsale t
            left join managers m on t.manager_id = m.id)

        select t.office, count(t.result) as cnt_call,
        (count(t.result) filter (where t.result = 'yes')::float / count(t.result)) as convers
        from count_result t
            group by t.office
    )

    select t.office, 
    avg(t.convers) over (partition by t.office) as avg_convers
    from count_offices t
    order by avg_convers desc

'''
select(sql)

,office,avg_convers
0,1,1.000000
1,4,0.666667
2,5,0.500000
3,8,0.500000
4,7,0.000000


5.  Вывести минимальный порядковый номер звонка клиента с RESULT = Yes, перед которым был RESULT = No для каждого менеджера.
    - во временной таблице c функцией lag() получил столбец с предыдущими значениями и отсортированные значения номера звонка 
    - добавил столбец yes-no, в котором записывались значения 1 или 0 в зависимости от того, что находится в столбце result и в столбце с предыдущими значениями
    - выбрал столбцы с данными менеджеров и минимальным значением номера звонка и заджойнил с таблицей managers с необходимыми комбинациями result=yes and lag(result)=0

In [79]:
sql = '''
    with results as (
        select t.*, 
        lag(t.result) over (order by t.call_id) as prev_result,

        case when t.result = 'yes' and 
        (lag(t.result) over (order by call_id)) = 'no'
        then 1 else 0 end as yes_no
        from upsale t
    )

    select m.login, m.name, min(t.call_id) as min_call from results t
    left join managers m on t.manager_id = m.id
    where t.yes_no = 1
    group by m.login, m.name

'''

select(sql)

,login,name,min_call
0,aleensr,Allen D,5
1,duckduck,Muddy W,9
2,jool,Jolly A,7


6. Получить для каждого менеджера первое принятое предложение.
    - заджойнил 2 таблицы по условию result = 'yes'
    - с помощью функции row_number() отранжировал даты по каждому менеджеру
    - в полученной таблице выбрал первый ранг по каждому менеджеру

In [96]:
sql = '''

with first_time as (
    with all_table as (
        select m.*, t.* 
        from upsale t
            left join managers m on t.manager_id = m.id
            where t.result = 'yes'
    )
            select t.*, 
            row_number() over (partition by t.manager_id order by t.dt) as rnk
            from all_table t
)

select t.name, t.office, t.dt, t.client_id, t.call_id, t.factor_id
    from first_time t
    where t.rnk = 1

'''

select(sql)

,name,office,dt,client_id,call_id,factor_id
0,Hemp A,4,01.02.2017,45,1,1
1,Jolly A,5,03.02.2017,123,7,3
2,Hobert F,5,04.02.2017,678,11,5
3,Allen D,1,02.02.2017,678,5,4
4,Muddy W,8,03.02.2017,90,9,1


7.  Предложить запрос, показывающий ранее не использованные операторы SQL.
    1. Оператор IN Запрос по продажам менеджеров в нескольких офисах


In [27]:
sql = '''
    select m.*, t.* 
        from upsale t
            left join managers m on t.manager_id = m.id
        where office in (1, 5)

'''
select(sql)

,id,login,name,office,id,dt,manager_id,client_id,call_id,factor_id,result
0,5,aleensr,Allen D,1,5,02.02.2017,5,678,5,4,yes
1,5,aleensr,Allen D,1,12,04.02.2017,5,734,12,2,yes
2,3,hobert,Hobert F,5,2,01.02.2017,3,67,2,1,no
3,3,hobert,Hobert F,5,11,04.02.2017,3,678,11,5,yes
4,2,jool,Jolly A,5,7,03.02.2017,2,123,7,3,yes
5,2,jool,Jolly A,5,8,03.02.2017,2,45,8,2,no


7.  Предложить запрос, показывающий ранее не использованные операторы SQL.
    2. Оператор IS NULL Запрос по менеджерам у которых нет звонков

In [97]:
sql = '''
    select m.name, m.office, t.call_id 
        from upsale t
            full join managers m on t.manager_id = m.id
        where call_id is null
'''
select(sql)

,name,office,call_id
0,Goofy M,8,None


8.  Предложить запрос, который покажет интересную или полезную информацию из этих данных.

8.1 Количество контактов с клиентами и конверсия по факторам.
    - По количеству предложений клиентам лидирует 2-й фактор: 4 раза предлагался и только 1 раз удачно, конверсия 0,25. Удачная продажа принадлежит Allen D, который является лидером по результативности (100% конверсия). Можно было бы проверить гипотезу, что его легко предлагать, но он не интересен клиентам.
    - 3-й и 5-й факторы предлагаются редко, но по ним 100% продажи. Это может означать, что они интересны клиентам, но для менеджеров они представляют к.л. трудности, либо по данным факторам низкая мотивация и их неинтересно продавать.
    - 1-й и 4-й факторы хорошо продаются и по ним хорошая конверсия - 0,7. 



In [148]:
sql = '''
with all_tbl as (
    select m.*, t.* 
        from upsale t
            left join managers m on t.manager_id = m.id
    )

    select t.factor_id,
    count(t.factor_id) as cnt_factor,
    (count(t.result) filter (where t.result = 'yes')::float / count(t.result)) as convers
    from all_tbl t
    group by t.factor_id
    order by convers desc, t.factor_id
    '''
select(sql)

,factor_id,cnt_factor,convers
0,3,1,1.000000
1,5,1,1.000000
2,1,3,0.666667
3,4,3,0.666667
4,2,4,0.250000


8.2. Какие дни были результативные по конверсии.
    - худший день 1.02.2017: максимум продаж и минимум конверсии
    - лучший день - 04.02.2017 максимум конверсии и среднее значение продаж

In [129]:
sql = '''
select t.dt, count(t.result) as cnt_sale, 
    (count(t.result) filter (where t.result = 'yes')::float / count(t.result)) as convers
from upsale t
group by t.dt
order by convers
'''
select(sql)

,dt,cnt_sale,convers
0,01.02.2017,4,0.250000
1,02.02.2017,2,0.500000
2,03.02.2017,3,0.666667
3,04.02.2017,3,1.000000


In [11]:
sql = '''
    select m.*, t.* 
        from upsale t
            left join managers m on t.manager_id = m.id
'''
select(sql)

,id,login,name,office,id,dt,manager_id,client_id,call_id,factor_id,result
0,1,hemp,Hemp A,4,10,04.02.2017,1,893,10,4,yes
1,1,hemp,Hemp A,4,4,01.02.2017,1,45,4,4,no
2,1,hemp,Hemp A,4,1,01.02.2017,1,45,1,1,yes
3,2,jool,Jolly A,5,8,03.02.2017,2,45,8,2,no
4,2,jool,Jolly A,5,7,03.02.2017,2,123,7,3,yes
5,3,hobert,Hobert F,5,11,04.02.2017,3,678,11,5,yes
6,3,hobert,Hobert F,5,2,01.02.2017,3,67,2,1,no
7,4,frankjr,Gotty F,7,6,02.02.2017,4,890,6,2,no
8,5,aleensr,Allen D,1,12,04.02.2017,5,734,12,2,yes
9,5,aleensr,Allen D,1,5,02.02.2017,5,678,5,4,yes


In [40]:
sql = '''
    select m.*, t.* 
        from upsale t
            left join managers m on t.manager_id = m.id
        where 
            t.dt between '01.02.2017' and '03.02.2017'
'''
select(sql)

,id,login,name,office,id,dt,manager_id,client_id,call_id,factor_id,result
0,1,hemp,Hemp A,4,4,01.02.2017,1,45,4,4,no
1,1,hemp,Hemp A,4,1,01.02.2017,1,45,1,1,yes
2,2,jool,Jolly A,5,8,03.02.2017,2,45,8,2,no
3,2,jool,Jolly A,5,7,03.02.2017,2,123,7,3,yes
4,3,hobert,Hobert F,5,2,01.02.2017,3,67,2,1,no
5,4,frankjr,Gotty F,7,6,02.02.2017,4,890,6,2,no
6,5,aleensr,Allen D,1,5,02.02.2017,5,678,5,4,yes
7,7,duckduck,Muddy W,8,9,03.02.2017,7,90,9,1,yes
8,7,duckduck,Muddy W,8,3,01.02.2017,7,123,3,2,no


In [31]:
sql = '''
select localtime(1)
'''
select(sql)

,localtime
0,13:38:14.300000
